In [ ]:
import brightway2 as bw
import pandas as pd
import numpy as np

In [ ]:
bw.projects.set_current('GSA for paper')
co = bw.Database('CH consumption 1.0')
average_consump = co.search('average consumption')[0]

In [ ]:
list(average_consump.exchanges())

In [ ]:
list_functional_units = []
for exc in average_consump.exchanges():
    if exc['type'] != 'production':
        list_functional_units.append({exc['input']: exc['amount']})

# list_functional_units = []
# for act in list(co):
#     list_functional_units.append({act: 1})
        
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]

In [ ]:
bw.calculation_setups['av_consump_contribution_analysis'] = {'inv':list_functional_units, 'ia':list_methods}

In [ ]:
%%time
myMultiLCA = bw.MultiLCA('av_consump_contribution_analysis')

In [ ]:
lcia_unit = bw.Method(list_methods[0]).metadata['unit']

In [ ]:
fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_functional_units]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['exchanges'] = [list(fu.values())[0] for fu in list_functional_units]
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_functional_units]


In [ ]:
df.sort_values(lcia_unit, ascending=False, inplace=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

In [ ]:
# df_ = df.sort_values(lcia_unit)
# df_['kg CO2-Eq'].plot.barh(figsize=(20,40))

# Biosphere

In [ ]:
import bw2analyzer

In [ ]:
lca = bw.LCA({average_consump: 1}, list_methods[0])

In [ ]:
lca.lci()
lca.lcia()
lca.score

In [ ]:
ca = bw2analyzer.ContributionAnalysis()

In [ ]:
ca.annotated_top_processes(lca)

# Contribution analysis for sectors

In [ ]:
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]
list_fus = []
for cat_of_interest in category_activities:
    list_fus.append({co.search(cat_of_interest)[0]: 1})
    
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}

%%time
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')

lcia_unit = bw.Method(list_methods[0]).metadata['unit']

fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_fus]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]

df.sort_values(lcia_unit, ascending=False, inplace=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)